In [43]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM, Dropout, Dense, Input, Conv1D, MaxPooling1D, Bidirectional
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score, precision_score, recall_score, accuracy_score, r2_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [44]:
data = pd.read_csv(r"C:\Users\Manibharathi\Downloads\AI-based-Predictive-Maintenance\Data\live25_with_rul.csv")
data

,ENGINE_RUN_TINE (),ENGINE_RPM (),VEHICLE_SPEED (),THROTTLE (),ENGINE_LOAD (),COOLANT_TEMPERATURE (),LONG_TERM_FUEL_TRIM_BANK_1 (),SHORT_TERM_FUEL_TRIM_BANK_1 (),INTAKE_MANIFOLD_PRESSURE (),FUEL_TANK (),...,INTAKE_AIR_TEMP (),TIMING_ADVANCE (),CATALYST_TEMPERATURE_BANK1_SENSOR1 (),CATALYST_TEMPERATURE_BANK1_SENSOR2 (),CONTROL_MODULE_VOLTAGE (),COMMANDED_EVAPORATIVE_PURGE (),TIME_RUN_WITH_MIL_ON (),TIME_SINCE_TROUBLE_CODES_CLEARED (),DISTANCE_TRAVELED_WITH_MIL_ON (),RUL
0,0.0,0.0,17.647058,0.000000,26.0,-4.68750,0.00000,101.0,36.078430,49.803921,...,5.0,438.000000,300.799988,12.421,0.000000,0.0,8808.0,0.0,255.0,100.000000
1,0.0,0.0,17.647058,0.000000,26.0,-4.68750,0.00000,101.0,36.078430,49.803921,...,5.0,438.000000,300.799988,12.421,0.000000,0.0,8808.0,0.0,255.0,99.973712
2,0.0,0.0,19.607843,0.000000,26.0,-4.68750,0.00000,99.0,36.078430,49.803921,...,5.0,438.000000,300.799988,12.421,0.000000,0.0,8808.0,0.0,255.0,99.947424
3,0.0,0.0,19.607843,0.000000,26.0,-4.68750,0.00000,99.0,36.078430,49.803921,...,5.0,438.000000,300.799988,12.421,0.000000,0.0,8808.0,0.0,255.0,99.921136
4,0.0,0.0,19.607843,0.000000,26.0,-4.68750,0.00000,99.0,36.078430,52.156864,...,5.0,438.000000,300.799988,11.679,0.000000,0.0,8808.0,0.0,255.0,99.894848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3799,690.5,0.0,15.686275,25.490196,89.0,-0.78125,-0.78125,28.0,32.156864,47.450981,...,2.0,442.399994,345.799988,13.652,25.490196,0.0,8839.0,0.0,255.0,0.500000
3800,690.5,0.0,15.686275,25.490196,89.0,-0.78125,-0.78125,28.0,32.156864,47.450981,...,2.0,442.399994,345.799988,13.652,25.490196,0.0,8839.0,0.0,255.0,0.500000
3801,690.5,0.0,15.686275,25.490196,89.0,-0.78125,-0.78125,28.0,32.156864,47.450981,...,2.0,441.399994,344.700012,13.652,25.490196,0.0,8839.0,0.0,255.0,0.500000
3802,690.5,0.0,15.686275,25.490196,89.0,-0.78125,-0.78125,28.0,32.156864,47.450981,...,2.0,441.399994,344.700012,13.652,25.490196,0.0,8839.0,0.0,255.0,0.500000


In [45]:
# Normalize features

scaler = MinMaxScaler()
features = data[['ENGINE_RPM ()', 'ENGINE_LOAD ()', 'COOLANT_TEMPERATURE ()', 'CATALYST_TEMPERATURE_BANK1_SENSOR1 ()', 'CATALYST_TEMPERATURE_BANK1_SENSOR2 ()']]
features_scaled = scaler.fit_transform(features)
data

,ENGINE_RUN_TINE (),ENGINE_RPM (),VEHICLE_SPEED (),THROTTLE (),ENGINE_LOAD (),COOLANT_TEMPERATURE (),LONG_TERM_FUEL_TRIM_BANK_1 (),SHORT_TERM_FUEL_TRIM_BANK_1 (),INTAKE_MANIFOLD_PRESSURE (),FUEL_TANK (),...,INTAKE_AIR_TEMP (),TIMING_ADVANCE (),CATALYST_TEMPERATURE_BANK1_SENSOR1 (),CATALYST_TEMPERATURE_BANK1_SENSOR2 (),CONTROL_MODULE_VOLTAGE (),COMMANDED_EVAPORATIVE_PURGE (),TIME_RUN_WITH_MIL_ON (),TIME_SINCE_TROUBLE_CODES_CLEARED (),DISTANCE_TRAVELED_WITH_MIL_ON (),RUL
0,0.0,0.0,17.647058,0.000000,26.0,-4.68750,0.00000,101.0,36.078430,49.803921,...,5.0,438.000000,300.799988,12.421,0.000000,0.0,8808.0,0.0,255.0,100.000000
1,0.0,0.0,17.647058,0.000000,26.0,-4.68750,0.00000,101.0,36.078430,49.803921,...,5.0,438.000000,300.799988,12.421,0.000000,0.0,8808.0,0.0,255.0,99.973712
2,0.0,0.0,19.607843,0.000000,26.0,-4.68750,0.00000,99.0,36.078430,49.803921,...,5.0,438.000000,300.799988,12.421,0.000000,0.0,8808.0,0.0,255.0,99.947424
3,0.0,0.0,19.607843,0.000000,26.0,-4.68750,0.00000,99.0,36.078430,49.803921,...,5.0,438.000000,300.799988,12.421,0.000000,0.0,8808.0,0.0,255.0,99.921136
4,0.0,0.0,19.607843,0.000000,26.0,-4.68750,0.00000,99.0,36.078430,52.156864,...,5.0,438.000000,300.799988,11.679,0.000000,0.0,8808.0,0.0,255.0,99.894848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3799,690.5,0.0,15.686275,25.490196,89.0,-0.78125,-0.78125,28.0,32.156864,47.450981,...,2.0,442.399994,345.799988,13.652,25.490196,0.0,8839.0,0.0,255.0,0.500000
3800,690.5,0.0,15.686275,25.490196,89.0,-0.78125,-0.78125,28.0,32.156864,47.450981,...,2.0,442.399994,345.799988,13.652,25.490196,0.0,8839.0,0.0,255.0,0.500000
3801,690.5,0.0,15.686275,25.490196,89.0,-0.78125,-0.78125,28.0,32.156864,47.450981,...,2.0,441.399994,344.700012,13.652,25.490196,0.0,8839.0,0.0,255.0,0.500000
3802,690.5,0.0,15.686275,25.490196,89.0,-0.78125,-0.78125,28.0,32.156864,47.450981,...,2.0,441.399994,344.700012,13.652,25.490196,0.0,8839.0,0.0,255.0,0.500000


In [46]:

data = data.drop(columns=data.columns[data.isna().all()]).copy()  # Drop columns with all NaN values

# Reapply scaling after removing problematic columns
scaled_data = scaler.fit_transform(data.drop(columns=['RUL']))
rul_scaled = scaler.fit_transform(data[['RUL']])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(scaled_data, rul_scaled, test_size=0.2, shuffle=False)

# Use TimeSeriesGenerator to generate sequences for LSTM
sequence_length = 50
train_generator = TimeseriesGenerator(X_train, y_train, length=sequence_length, batch_size=40)
test_generator = TimeseriesGenerator(X_test, y_test, length=sequence_length, batch_size=40)

data

,ENGINE_RUN_TINE (),ENGINE_RPM (),VEHICLE_SPEED (),THROTTLE (),ENGINE_LOAD (),COOLANT_TEMPERATURE (),LONG_TERM_FUEL_TRIM_BANK_1 (),SHORT_TERM_FUEL_TRIM_BANK_1 (),INTAKE_MANIFOLD_PRESSURE (),FUEL_TANK (),...,INTAKE_AIR_TEMP (),TIMING_ADVANCE (),CATALYST_TEMPERATURE_BANK1_SENSOR1 (),CATALYST_TEMPERATURE_BANK1_SENSOR2 (),CONTROL_MODULE_VOLTAGE (),COMMANDED_EVAPORATIVE_PURGE (),TIME_RUN_WITH_MIL_ON (),TIME_SINCE_TROUBLE_CODES_CLEARED (),DISTANCE_TRAVELED_WITH_MIL_ON (),RUL
0,0.0,0.0,17.647058,0.000000,26.0,-4.68750,0.00000,101.0,36.078430,49.803921,...,5.0,438.000000,300.799988,12.421,0.000000,0.0,8808.0,0.0,255.0,100.000000
1,0.0,0.0,17.647058,0.000000,26.0,-4.68750,0.00000,101.0,36.078430,49.803921,...,5.0,438.000000,300.799988,12.421,0.000000,0.0,8808.0,0.0,255.0,99.973712
2,0.0,0.0,19.607843,0.000000,26.0,-4.68750,0.00000,99.0,36.078430,49.803921,...,5.0,438.000000,300.799988,12.421,0.000000,0.0,8808.0,0.0,255.0,99.947424
3,0.0,0.0,19.607843,0.000000,26.0,-4.68750,0.00000,99.0,36.078430,49.803921,...,5.0,438.000000,300.799988,12.421,0.000000,0.0,8808.0,0.0,255.0,99.921136
4,0.0,0.0,19.607843,0.000000,26.0,-4.68750,0.00000,99.0,36.078430,52.156864,...,5.0,438.000000,300.799988,11.679,0.000000,0.0,8808.0,0.0,255.0,99.894848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3799,690.5,0.0,15.686275,25.490196,89.0,-0.78125,-0.78125,28.0,32.156864,47.450981,...,2.0,442.399994,345.799988,13.652,25.490196,0.0,8839.0,0.0,255.0,0.500000
3800,690.5,0.0,15.686275,25.490196,89.0,-0.78125,-0.78125,28.0,32.156864,47.450981,...,2.0,442.399994,345.799988,13.652,25.490196,0.0,8839.0,0.0,255.0,0.500000
3801,690.5,0.0,15.686275,25.490196,89.0,-0.78125,-0.78125,28.0,32.156864,47.450981,...,2.0,441.399994,344.700012,13.652,25.490196,0.0,8839.0,0.0,255.0,0.500000
3802,690.5,0.0,15.686275,25.490196,89.0,-0.78125,-0.78125,28.0,32.156864,47.450981,...,2.0,441.399994,344.700012,13.652,25.490196,0.0,8839.0,0.0,255.0,0.500000


In [47]:
# Define CNN-LSTM model
import random
def create_cnn_lstm_model(learning_rate=0.001, dropout_rate=0.2, lstm_units=50, conv_filters=64, kernel_size=3):
    model = Sequential()
    model.add(Input(shape=(sequence_length, X_train.shape[1])))  # Adjusted shape for TimeseriesGenerator output
    model.add(Conv1D(filters=conv_filters, kernel_size=kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Bidirectional(LSTM(units=lstm_units, activation='tanh', return_sequences=True)))
    model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(units=lstm_units, activation='tanh', return_sequences=False)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))  # Output layer for RUL prediction
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

# Define hyperparameter values to search
learning_rates = [0.001, 0.0005, 0.0001]
dropout_rates = [0.2, 0.3, 0.4]
lstm_units_list = [50, 64, 128]
conv_filters_list = [32, 64, 128]
kernel_sizes = [3, 5]

# Generate a random sample of hyperparameter combinations
n_trials = 10  # Define the number of random trials
random.seed(42)
hyperparameter_combinations = [
    (lr, dr, lstm_units, conv_filters, kernel_size)
    for lr in learning_rates
    for dr in dropout_rates
    for lstm_units in lstm_units_list
    for conv_filters in conv_filters_list
    for kernel_size in kernel_sizes
]
random_trials = random.sample(hyperparameter_combinations, n_trials)

# Tracking the best model and best loss
best_model = None
best_loss = float('inf')

# Hyperparameter tuning loop with random sampling
for lr, dr, lstm_units, conv_filters, kernel_size in random_trials:
    print(f"Training with lr={lr}, dropout={dr}, lstm_units={lstm_units}, conv_filters={conv_filters}, kernel_size={kernel_size}")
    model = create_cnn_lstm_model(
        learning_rate=lr,
        dropout_rate=dr,
        lstm_units=lstm_units,
        conv_filters=conv_filters,
        kernel_size=kernel_size
    )
    
    # Train the model (use fewer epochs initially to save time)
    model.fit(train_generator, validation_data=test_generator, epochs=10, verbose=0)
    
    # Evaluate the model on the test data
    loss = model.evaluate(test_generator, verbose=0)
    
    # Track the best model based on validation loss
    if loss < best_loss:
        best_loss = loss
        best_model = model

print(f"Best model loss: {best_loss}")


Training with lr=0.001, dropout=0.3, lstm_units=64, conv_filters=128, kernel_size=3


C:\Users\Manibharathi\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Training with lr=0.001, dropout=0.2, lstm_units=64, conv_filters=32, kernel_size=3
Training with lr=0.0005, dropout=0.2, lstm_units=128, conv_filters=128, kernel_size=3
Training with lr=0.0005, dropout=0.2, lstm_units=64, conv_filters=64, kernel_size=3
Training with lr=0.0005, dropout=0.2, lstm_units=50, conv_filters=64, kernel_size=5
Training with lr=0.001, dropout=0.3, lstm_units=128, conv_filters=128, kernel_size=5
Training with lr=0.001, dropout=0.3, lstm_units=64, conv_filters=64, kernel_size=3
Training with lr=0.0001, dropout=0.3, lstm_units=128, conv_filters=32, kernel_size=5
Training with lr=0.001, dropout=0.3, lstm_units=50, conv_filters=128, kernel_size=3
Training with lr=0.0001, dropout=0.4, lstm_units=64, conv_filters=32, kernel_size=5
Best model loss: 0.0002909385657403618


In [ ]:
y_pred = []
y_pred = best_model.predict(test_generator)

# Convert predictions and true values to 1D arrays for metric calculation
y_test_actual = np.array([y[0] for y in y_test[sequence_length:]]
                        )  # Remove initial steps to match sequence length
y_pred_actual = y_pred.flatten()

# Calculate MAE
mae = mean_absolute_error(y_test_actual, y_pred_actual)
print(f"Mean Absolute Error (MAE): {mae}")

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test_actual, y_pred_actual))
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Calculate R² Score
r2 = r2_score(y_test_actual, y_pred_actual)
print(f"R² Score: {r2}")